# Who is NOT using the LMS?
This question came from the head of Faculty Development. It was just after Spring Break, with one extra week for Faculty to shift. 
* Which courses aren't being used?
* How many students are affected?
* Who is teaching these courses?
* What department are they in?


The query: 
```sql
SELECT SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, '-[A-Z&]+'),2) COURSE_UNIT,
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, ' [A-Z]{3}$'),2) COURSE_TYPE,
       REPLACE(cm.COURSE_NAME,',',' ') COURSE_NAME,
       cm.COURSE_ID,
       cm.AVAILABLE_IND,
       MAX(cu.LAST_ACCESS_DATE) last_access,
       SUM(students.students)/COUNT(students.students) student_count,
       listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
       listagg(u.firstname || ' ' || u.LASTNAME, ';') WITHIN GROUP (order by u.email) names

FROM BB_BB60.USERS u inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.pk1
left join (select crsmain_pk1, count(pk1) students
    from bb_bb60.COURSE_USERS
    where role='S'
    and row_status = 0
    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1

WHERE cm.COURSE_NAME LIKE '2020' || :season ||'-%' AND (cu.ROLE='P' OR cu.ROLE='T') AND
      NOT EXISTS (SELECT cc.crsmain_pk1 course_id
      from bb_bb60.COURSE_COURSE cc WHERE cc.CRSMAIN_PK1 = cm.pk1)

GROUP BY cm.COURSE_NAME, cm.COURSE_ID, cm.AVAILABLE_IND
HAVING SUM(students.students)/COUNT(students.students) IS NOT NULL
ORDER BY student_count desc
```

In [3]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

### Which courses aren't being used?

In [5]:
query = """
SELECT cm.COURSE_ID, MAX(cu.LAST_ACCESS_DATE) last_access
FROM BB_BB60.USERS u 
    inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
    inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.PK1
    WHERE cm.COURSE_NAME LIKE '2020Spring-%' AND (cu.ROLE='P' OR cu.ROLE='T')
GROUP BY cm.COURSE_ID
"""

pd.read_sql(query, con=engine).head()

,course_id,last_access
0,4202-40893,NaT
1,4202-41678,NaT
2,4202-52073,2020-06-16 13:21:21
3,4202-55739,2020-05-15 13:47:13
4,4202-56906,NaT


### Problems

* Course_ID doesn't have much information for sorting.
* Not sure who to contact for each course.

In [10]:
query = """
SELECT cm.COURSE_ID, 
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, '-[A-Z&]+'),2) COURSE_UNIT,
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, ' [A-Z]{3}$'),2) COURSE_TYPE,
       REPLACE(cm.COURSE_NAME,',',' ') COURSE_NAME,MAX(cu.LAST_ACCESS_DATE) last_access,
       listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
       listagg(u.firstname || ' ' || u.LASTNAME, ';') WITHIN GROUP (order by u.email) names

FROM BB_BB60.USERS u 
    inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
    inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.PK1
    WHERE cm.COURSE_NAME LIKE '2020Spring-%' AND (cu.ROLE='P' OR cu.ROLE='T')

GROUP BY cm.COURSE_ID, cm.COURSE_NAME
"""

pd.read_sql(query, con=engine).head()

,course_id,course_unit,course_type,course_name,last_access,emails,names
0,4202-40001,AAAS,LEC,2020Spring-AAAS 306 The Black Experience in th...,2020-05-19 15:56:34,rmjelks@ku.edu,Randal Jelks
1,4202-40002,AAAS,IND,2020Spring-AAAS 400 Readings in: _____ IND,NaT,macgonag@ku.edu,Elizabeth MacGonagle
2,4202-40003,AAAS,IND,2020Spring-AAAS 400 Readings in: _____ IND,NaT,nboussofara@ku.edu,Naima Boussofara
3,4202-40004,AAAS,IND,2020Spring-AAAS 400 Readings in: _____ IND,NaT,dpen@ku.edu,Dorthy Pennington
4,4202-40005,AAAS,IND,2020Spring-AAAS 400 Readings in: _____ IND,NaT,ukpokodu@ku.edu,Peter Ukpokodu
